# Markov Process

## Markov Property
In a stochastic process, the state at time $t$ is $s_t \in \mathbb{S}$. Usually, $s_t$ only depends on its previous states {$s_0$, $s_1$, $s_2$, ..., $s_{t-1}$}. If further we have $s_t$ only depends on its last previous state $s_{t-1}$, we call this stochastic process to have Markov property, namely $$P(s_t|s_0, s_1, ..., s_{t-1}) = P(s_t|s_{t-1}).$$

## Markov Process
Markov process represents the stochastic processes with Markov property. Now that we know the current states only depend on a transition distribution between its latest previous states, the whole Markov process can be represented by a tuple $<\mathcal{S}, \mathcal{P}>$, where $\mathcal{S}$ is the set of all possible states, and $\mathcal{P}$ is a state transition matrix that defines the transition probability between every pair of the states in $\mathcal{S}$: $$\mathcal{P} = \begin{bmatrix}
P(s_1|s_1) & \cdots & P(s_n|s_1) \\
\vdots & \ddots & \vdots \\
P(s_1|s_n) & \cdots & P(s_n|s_n)
\end{bmatrix}$$

## Markov Rewrad Process
We may add reward function $r$ and discount factor $\gamma$ to Markov Process to get a Markov Reward Process $<\mathcal{S}, \mathcal{P}, \mathcal{r}, \mathcal{\gamma}>$. $r(s)$ intakes a state $s$ and returns the expectation of reward when the process state transfers into $s$. $\gamma \in [0, 1)$ discounts for the uncertainty of further rewards. A $\gamma$ closer to 1 focuses more on further cumulative profit while closer to 0 cares more about recent rewards.

### Return
In a Markov Reward Process (MRP), return $G_t$ is the cumulative discounted reward since time $t$ to the end of the process (Notice that we are using $r$ for the expected reward, and $R$ for the acutal reward). $$G_t = R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + ... = \sum_{k=0}^\infty \gamma^{k}R_{t+k}$$

Below shows how you may calculate the return in an MPR.

In [2]:
import numpy as np
np.random.seed(0)
# 定义状态转移概率矩阵P
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0]  # 定义奖励函数
gamma = 0.5  # 定义折扣因子


# 给定一条序列,计算从某个索引（起始状态）开始到序列最后（终止状态）得到的回报
def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = gamma * G + rewards[chain[i] - 1]
    return G


# 一个状态序列,s1-s2-s3-s6
chain = [1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print("根据本序列计算得到回报为：%s。" % G)

根据本序列计算得到回报为：-2.5。


### Value function
In an MPR, the expected return of a state is called the value of that state. The values of all states composes the value function $V$. 

$V(s)=\mathbb{E}[G_t|S_t=s] \\ \quad = \mathbb{E}[R_t + \gamma R_{t+1} + \gamma^2 R_{t+2} + ... |S_t=s]\\ \quad = \mathbb{E}[R_t + \gamma (R_{t+1} + \gamma R_{t+2} + ... )|S_t=s] \\ \quad = \mathbb{E}[R_t + \gamma G_{t+1}|S_t=s] \\ \quad = \mathbb{E}[R_t|S_t=s] + \mathbb{E}[\gamma G_{t+1}|S_t=s] \\ \quad = r(s) + \gamma \mathbb{E}[V(S_{t+1})|S_t=s]$.

Since the transformation matrix defines how states are transformed, we have:$$V(s) = r(s) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s)V(s'),$$

which is the **Bellman equation** of MRP.

Assume we have $n$ states in an MPR, we can write the values for all the states as a column vector $\mathcal{V} = [V(s_1), V(s_2), ..., V(s_n)]^T$. Similarly, we can have the column vecotr of reward $\mathcal{R} = [r(s_1), r(s_2), ..., r(s_n)]^T$. Therefore, we got hte matrix format of Bellman equation: $$\mathcal{V} = \mathcal{R} + \gamma \mathcal{P} \mathcal{V}$$.

Solve the equation, we get the analytical solution of Bellman equation: $$\mathcal{V} = (I - \gamma \mathcal{P})^{-1} \mathcal{R}$$.

The above analytical solution have time complexity of $O(n^3)$, so it only fits in small-size MRPs. The following code calculates for the analytical solution.

In [3]:
def compute(P, rewards, gamma, states_num):
    ''' 利用贝尔曼方程的矩阵形式计算解析解,states_num是MRP的状态数 '''
    rewards = np.array(rewards).reshape((-1, 1))  #将rewards写成列向量形式
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P),
                   rewards)
    return value


V = compute(P, rewards, gamma, 6)
print("MRP中每个状态价值分别为\n", V)

MRP中每个状态价值分别为
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


## Markov Decision Process
MP and MRP are automatic processes. If there exists a stimulus from the environment that works together on the process, we call it Markov Decision Process (MDP). This stimulus is call the action of the agent. Add actions into MRP gives us MDP $<\mathcal{S}, \mathcal{A}, \mathcal{P}, r, \gamma>$, where $\mathcal{A}$ is the set of all possible actions. Accordingly, $r$ now refers to $r(s, a)$, a reward that depends on both the state and the action, and $\mathcal{P}(s'|s,a)$ describes how the action and the current states together points to the next state.

### Policy
Policy $\pi$ of a agent describes how the agent take actions. $\pi(a|s) = P(A_t=a|S_t=s)$ is the probability of the agent taking action $a$ when the current state is $s$. 

A policy could be deterministic when it has a one-hot distribution, or it could be stochastic where every time the agent samples from the distribution. 

### State Value Function
We use $V^{\pi}(s)$ to describe the state value at $s$ given the policy $\pi$ in MDP. $$V^{\pi}(s) = \mathbb{E}_{\pi} [G_t | S_t = s]$$

### Action Value Function
We use $Q^{\pi}(s, a)$ to describe the action $a$'s value in state $s$ given policy $\pi$. $$Q^{\pi}(s, a) = \mathbb{E}_{\pi} [G_t|S_t=s, A_t=a]$$.

We can observe that the state salue function is a marginalization of the action value function: $$V^{\pi}(s) = \sum_{a \in \mathcal{A}} \pi(a|s)Q^{\pi}(s, a)$$

Accordingly, we can observe that in MDP, the action value equals the reward of current state-action pair plus the discounted weighted sum of all possible state values weighted by their transformation probability: $$Q^{\pi}(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s'|s,a)V(s')$$

## Bellman Expectation Equation
Simple inference brings us to the Bellman expectation equations:
$$V^{\pi}(s) = \mathbb{E}_{\pi} [R_t + \gamma V^\pi(S_{t+1})| S_t = s] \\ \quad = \sum_{a \in \mathcal{A}} \pi(a|s) (r(s, a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s, a)V^{\pi}(s'))$$

$$Q^{\pi}(s, a) = \mathbb{E}_{\pi} [R_t + \gamma Q^{\pi}(S_{t+1}, A_{t+1})|S_t=s, A_t=a] \\ \quad = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} p(s'|s, a)\sum_{a \in \mathcal{A}} \pi(a'|s')Q^{\pi}(s', a')$$

## Marginalization of $r$ and $P$
Marginalization of $r$ and $P$ gives us a way to transform MDP to MRP. Therefore, we can use the same analytical solution do solve MDP. 

$$r'(s) = \sum_{a \in \mathcal{A}} \pi(a|s)r(s, a)$$

$$P'(s'|s) = \sum_{a \in \mathcal{A}} \pi(a|s)P(s'|s, a)$$

The code below shows the analytical solution of our MDP.

In [4]:
gamma = 0.5
# 转化后的MRP的状态转移矩阵
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


# Monte Carlo Methods
Monte Carlo methods estimates by random repetitive draws. Let us see how we can use Monte Carlo methods to estimate the value functions of an MDP. 

To do so, we can sample several sequences from a state in MDP, and use the average of the returns of all the sequences to be the value estimation, i.e., $V^{\pi}(s) = \frac{1}{N} \sum_{i=1}^N G_t^{(i)}$.

Notice that we only need to calculate the values for unqiue sequences, and counts the appearances of each unique sequence in lieu of calcuate per sampling. 

To further accelerate, we may use the incremental update for the average: $N(s) \leftarrow N(s) + 1; V(s) \leftarrow V(s) + \frac{1}{N(s)} (G_t^{(i)}-V(s))$.

In [5]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}


# 把输入的两个字符串通过“-”连接,便于使用上述定义的P、R变量
def join(str1, str2):
    return str1 + '-' + str2

In [29]:
def sample(MDP, Pi, timestep_max, number):
    ''' 采样函数,策略Pi,限制最长时间步timestep_max,总共采样序列数number '''
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        # TODO: fill in the logic to sample an episode
        timestep = 0 
        # sample a state as the staring state
        s0 = np.random.choice(S[:-1])
        s_cur = s0
        while s_cur != "s5" and timestep <= timestep_max:
            timestep += 1

            # choose an action
            rand = np.random.rand()
            temp = 0
            a = None
            for a_prime in A:
                temp += Pi.get(join(s_cur, a_prime), 0) # assign prob=0 for all impossible actions
                if temp > rand:
                    a = a_prime
                    r = R.get(join(s_cur, a))
                    break
            
            # get to next state
            rand = np.random.rand()
            temp = 0
            s_next = None
            for s_prime in S:
                temp += P.get(join(join(s_cur, a), s_prime), 0)
                if temp > rand:
                    s_next = s_prime
                    break
            episode.append((s_cur, a, r, s_next))
            s_cur = s_next

        episodes.append(episode)
    return episodes


# 采样5次,每个序列最长不超过20步
episodes = sample(MDP, Pi_1, 20, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])

第一条序列
 [(np.str_('s4'), '前往s5', 10, 's5')]
第二条序列
 [(np.str_('s3'), '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]
第五条序列
 [(np.str_('s4'), '前往s5', 10, 's5')]


In [38]:
# 对所有采样序列计算所有状态的价值
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        # TODO: calculate for each episode
        for s, a, r, s_prime in episode[::-1]: #一个序列从后往前计算
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]


timestep_max = 20
# 采样1000次,可以自行修改
episodes = sample(MDP, Pi_1, timestep_max, 1000)
gamma = 0.5
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma)
print("使用蒙特卡洛方法计算MDP的状态价值为\n", V)

使用蒙特卡洛方法计算MDP的状态价值为
 {'s1': -1.2125020454474666, 's2': -1.6560549844853067, 's3': 0.5462728886332628, 's4': 6.2321596775902135, 's5': 0}


# Occupancy Measure

Different policies affects the distribution of actions, thus affecting the distributions of states the agent could access. Therefore, the policy actually affects the value function. To measure such difference, we introduce the occupancy measure.

Define the initial state distribution as $\nu_0(s)$, $P^{\pi}_t(s)$ as the probability of the agent acces state $s$ at time $t$ under policy $\pi$, $P^{\pi}_0(s) = \nu_0(s)$. Then we can define the state visitation of a policy as $$P^{\pi}_t(s) = (1-\gamma) \sum_{t=0}^\infty \gamma^t P^{\pi}_t(s),$$ where $(1-\gamma)$ is a normalization factor to make the probability of the summation equals 1.

Theoratically, to calculate the state visitation probability, we will need to go through infinty steps. However, it also works for a environment where the agent stops after finite steps (which is usually the case).

$P^{\pi}_t(s)$ has the following property: $$P^{\pi}_t(s) = (1-\gamma)\nu_0(s) + \gamma \int P(s'|s,a)\pi(a|s)\nu^\pi(s) ds da$$

The occupancy measure $\rho^\pi(s, a)$ represents probability of the pair $(s, a)$ being visited: $$\rho^\pi(s, a) = (1-\gamma) \sum_{t=0}^\infty \gamma^t P^{\pi}_t(s)\pi(a|s)$$.

Furthermore, we have two theorems:

Theorem 1: $\rho^{\pi_1} = \rho^{\pi_1} \Leftrightarrow \pi_1 = \pi_2$.

Theorem 2: given a valid occupancy measure $\rho$, the only policy that has occupancy measure $\rho$ is $\pi_\rho = \frac{\rho(s, a)}{\sum_{a'}\rho(s, a')}$

The following code estimates $\rho$ by the frequency of $(s, a)$ appears.

In [50]:
def occupancy(episodes, s, a, timestep_max, gamma):
    ''' 计算状态动作对（s,a）出现的频率,以此来估算策略的占用度量 '''
    rho = 0
    total_times = np.zeros(timestep_max)  # 记录每个时间步t各被经历过几次
    occur_times = np.zeros(timestep_max)  # 记录(s_t,a_t)=(s,a)的次数
    
    # TODO: fill up the code to estimate rho
    for episode in episodes:
        for i, (s_cur, a_cur, r, s_next) in enumerate(episode):
            total_times[i] += 1
            if s == s_cur and a == a_cur:
                occur_times[i] += 1
    for i, (tot, occ) in enumerate(zip(total_times, occur_times)):
        if occ > 0:
            rho += gamma**i * occ / tot


    return (1 - gamma) * rho


gamma = 0.5
timestep_max = 1000

episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

0.11424850994366208 0.239521095567568


# Optimal Policy

The object of RL mostly focuses on finding a policy that makes the agent get the maximal expected return starting from the initial state. To begin with, we difne the order between policies: $\pi > \pi' \iff \forall s \in \mathcal{S}, V^{\pi}(s) \geq V^{\pi'}(s)$. There could be multiple optimal policies, we represent them all as $\pi*(s)$.

According to Theorem 2 in Occupancy Measure, we know all the optimal policies must have the same state value function, which we call the optimal state value function: 

$V^*(s) = \max_\pi V^\pi(s), \forall s \in \mathcal{S}$. 

Similarly, we can define the optimal action value function:

$Q^*(a, s) = \max_\pi Q^{\pi}(a, s), \forall s \in \mathcal{S}, a \in \mathcal{A}$.

To make $Q^\pi(s, a)$ maximal, it is obvious that we should take the optimal policy at and after the current state, which gives us the relationship between $Q^\pi(s, a)$ and $V^*(s)$: $$Q^*(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}}P(s'|s, a)V^*(s')$$

On the other hand, $$V^*(s) = \max_{a \in \mathcal{A}} Q^*(s, a)$$

### Bellman Optimality Equation
If we replace $V^*(s)$ and $Q^*(s, a)$ in the equations above, we get the Bellman optimality quation: $$V^*(s) = \max_{a \in \mathcal{A}}\{r(s, a)\ + \gamma \sum_{s' \in \mathcal{S}} P(s'|s, a)V^*(s')\}$$ and $$Q^*(s, a) = r(s, a) + \gamma \sum_{s' \in \mathcal{S}} P(s'|s, a) \max_{a' \in \mathcal{A}} Q^*(s', a')$$